# Lab 6.2: Linear Regression

**Question 1**  

Using the cars data,
1) Fit a simple linear regression to predict `mpg` using `weight`.  

In [2]:
%pylab inline

import pandas as pd
import statsmodels.api as sm
import yaml

from seaborn import pairplot
from sqlalchemy import create_engine

pg_creds = yaml.load(open('../../pg_creds.yaml'))['student']

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**pg_creds))

cars = pd.read_sql("SELECT * FROM cars WHERE mpg IS NOT NULL;", engine, index_col='index')

Populating the interactive namespace from numpy and matplotlib


In [3]:
X = cars.weight
X = sm.add_constant(X)
y = cars.mpg

model = sm.OLS(y,X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.692
Model:                            OLS   Adj. R-squared:                  0.691
Method:                 Least Squares   F-statistic:                     888.9
Date:                Thu, 06 Oct 2016   Prob (F-statistic):          2.97e-103
Time:                        13:51:51   Log-Likelihood:                -1148.4
No. Observations:                 398   AIC:                             2301.
Df Residuals:                     396   BIC:                             2309.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         46.3174      0.795     58.243      0.000        44.754    47.881
weight        -0.0077      0.000    -29.814      0.000        -0.008    -0.007
==============================================================================
Omnibus:                       40.423   Durbin-Watson:                   0.850
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               56.695
Skew:                           0.713   Prob(JB):                     4.89e-13
Kurtosis:                       4.176   Cond. No.                     1.13e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.13e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

$$\hat{y} = b_0 + b_1 x $$  

$$ \hat{mpg} = 46.3174 -0.0077 (weight) $$

2) Comment on the model fit.  

R^2 = SSR/SST = 0.692

Since the R^2 is high, it means that ~69% of the variability in mpg is explained in the variability in weight.

3) Interpret the model. 

For every 1 unit increase in weight, the mpg decreases by .0077

4) Is `weight` useful for predicting `mpg`? Carry out a formal hypothesis test to show it.  

$h_0 : \beta_1 = 0$ - in other words, weight is not useful for predicting mpg

$h_a : \beta_1 != 0$

$\alpha = .05$

the p value is:	2.97e-103, so we reject the null hypothesis

5) Make a prediction for the average `mpg` of all cars that have a weight of 2000.  

In [4]:
mpg_hat = 46.3174 -0.0077 *(2000)
mpg_hat

30.9174

6) Make a prediction for a particular car that has a weight of 2000.  

In [5]:
# it's the same whether we want to calculate one object or the average - they have the same mean, but the variance is smaller for the average
mpg_hat = 46.3174 -0.0077 *(2000)
mpg_hat

30.9174

7) Write a Python function to calculate the confidence interval for your prediction in part 5).  

In [6]:
x = cars.weight
se = sqrt(results.mse_resid)
b0, b1 = results.params

x_new = 2000

def confidence_se(s_e, x, x_new):
    mean_x = x.mean()
    var_x = x.var()
    n = len(x)
    return s_e * (1/n + (x_new - mean_x)**2 / ((n - 1) * var_x))**0.5

sign = array([-1., 1.])
b0 + b1 * x_new + sign * 1.96 * confidence_se(se, x, x_new)

array([ 30.31449246,  31.61379612])



confidence interval of average means that when you're trying to predict mean mpg when weight = 2000, the mean estimated weight falls within the range given 95 percent of the time.

8) Write a Python function to calculate the prediction interval for your prediction in part 6).  

In [7]:
def prediction_se(s_e, x, x_new):
    mean_x = x.mean()
    var_x = x.var()
    n = len(x)
    return s_e * (1 + 1/n + (x_new - mean_x)**2 / ((n - 1) * var_x))**0.5

b0 + b1 * x_new + sign * 1.96 * prediction_se(se, x, x_new)

array([ 22.42392803,  39.50436055])


prediction interval means that when you're trying to predict mpg when weight = 2000, the estimated weight falls within the range given 95 percent of the time.

9) What are the differences between the intervals you found in parts 7) and 8)?

the prediction interval is always wider than the confidence interval for the average, because the prediction interval adds a 1.

An interval on the mean of the values is smaller than for a point, because have more data points.


**Question 2**  

You are shopping for a laptop computer at Best Buy. To help you with your decision, you decide to construct a regression model to predict the selling price of the laptop. The table `laptops` provides the following data for a random sample of laptops on Best Buy’s Web site:  

* Selling price
* Brand
* Screen size (in.)
* Hard drive size (GB)
* Amount of RAM memory (GB)
* Number of USB ports
* Weight (oz.) 

a) Using multiple regression, model selling price using the variables screen size, hard drive size, amount of ram, number of usb ports and weight.  

In [8]:
laptops = pd.read_sql("SELECT * FROM laptops", engine)

laptops.head()

,Price ($),Screen Size (in.),RAM Memory (GB),Hard drive (GB),USB Ports,Brand,Weight (oz.)
0,830,13.3,4,500,3,Toshiba,4.9
1,750,13.3,4,640,3,Toshiba,3.2
2,1200,11.6,2,128,2,Apple,2.3
3,1600,18.4,6,640,4,Toshiba,9.7
4,1900,18.4,8,500,4,Toshiba,9.7


In [9]:
print(laptops.columns)

Index(['Price ($)', 'Screen Size (in.)', 'RAM Memory (GB)', 'Hard drive (GB)',
       'USB Ports', 'Brand', 'Weight (oz.)'],
      dtype='object')


In [10]:
X2 = laptops[['Screen Size (in.)', 'Hard drive (GB)', 'RAM Memory (GB)', 'USB Ports', 'Weight (oz.)']]
X2 = sm.add_constant(X2)
y2 = laptops['Price ($)']

model2 = sm.OLS(y2, X2)
results2 = model2.fit()
results2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Price ($)   R-squared:                       0.117
Model:                            OLS   Adj. R-squared:                  0.040
Method:                 Least Squares   F-statistic:                     1.514
Date:                Thu, 06 Oct 2016   Prob (F-statistic):              0.200
Time:                        13:51:52   Log-Likelihood:                -477.99
No. Observations:                  63   AIC:                             968.0
Df Residuals:                      57   BIC:                             980.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
const               761.4987    946.954      0.804      0.425     -1134.744  2657.741
Screen Size (in.)     4.1113     96.206      0.043      0.966      -188.539   196.761
Hard drive (GB)       0.6561      0.459      1.429      0.159        -0.263     1.576
RAM Memory (GB)      12.8642     74.411      0.173      0.863      -136.141   161.870
USB Ports          -206.5346    123.563     -1.671      0.100      -453.965    40.896
Weight (oz.)         51.6251     99.210      0.520      0.605      -147.040   250.290
==============================================================================
Omnibus:                        9.835   Durbin-Watson:                   1.792
Prob(Omnibus):                  0.007   Jarque-Bera (JB):               10.384
Skew:                           0.993   Prob(JB):                      0.00556
Kurtosis:                       3.102   Cond. No.                     8.40e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.4e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

b) Perform and interpret the overall F test.  

$h_0 : \beta_1 =...=
\beta_n= 0$ - in other words, factors is not useful for predicting price

$h_a :$ at least 1 $\beta$ is not 0

$\alpha = .05$

Prob (F-statistic):	0.200

Since the p value is greater than .05, we fail to reject the null hypothesis, so model is not useful.

c) Using p-values, which variables appear to be needed in the model? Justify your answer.   

we want the variables that have p<0.05, so those are none of them.

d) Now create a new predictor that contains random numbers drawn from your favorite distribution, and include this predictor in your multiple regression model. Comment on the model fit. How does the new $R^2$ compare to the one in part a)?  

In [11]:
# Define a Poisson distribution with lambda = 2
# poisson2 = stats.poisson(2)

sim = numpy.random.poisson(1000, 63)

sim

array([ 908, 1000, 1009,  994, 1024, 1008,  957, 1005,  999,  983, 1020,
       1040,  963,  980, 1045,  966,  979, 1045, 1026, 1010,  949,  962,
        972, 1020, 1013, 1036, 1016,  993, 1004, 1010,  928,  973,  978,
       1023,  961, 1062,  986,  979, 1012,  975,  973,  990,  943,  993,
       1021, 1072, 1010,  959,  970,  988,  977, 1006, 1031, 1007,  987,
       1017, 1009, 1020,  967, 1008, 1041, 1033, 1002])

In [12]:
X3 = X2
X3['sim'] = sim
# print(type(X2))
y3 = laptops['Price ($)']

model3 = sm.OLS(y3, X3)
results3 = model3.fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Price ($)   R-squared:                       0.196
Model:                            OLS   Adj. R-squared:                  0.110
Method:                 Least Squares   F-statistic:                     2.271
Date:                Thu, 06 Oct 2016   Prob (F-statistic):             0.0495
Time:                        13:51:52   Log-Likelihood:                -475.06
No. Observations:                  63   AIC:                             964.1
Df Residuals:                      56   BIC:                             979.1
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
const             -3390.8410   1996.770     -1.698      0.095     -7390.851   609.169
Screen Size (in.)   -64.0953     97.133     -0.660      0.512      -258.676   130.486
Hard drive (GB)       0.7276      0.443      1.641      0.106        -0.160     1.616
RAM Memory (GB)      11.1347     71.662      0.155      0.877      -132.421   154.690
USB Ports          -209.4957    118.998     -1.760      0.084      -447.877    28.886
Weight (oz.)        115.2553     99.342      1.160      0.251       -83.750   314.261
sim                   4.8256      2.064      2.338      0.023         0.690     8.961
==============================================================================
Omnibus:                        6.886   Durbin-Watson:                   1.771
Prob(Omnibus):                  0.032   Jarque-Bera (JB):                6.867
Skew:                           0.808   Prob(JB):                       0.0323
Kurtosis:                       2.954   Cond. No.                     3.69e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.69e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

the simulated values drawn from the poisson distrubition also make for a bad model fit.  

old R^2 was: 0.117
new R^2 is: 0.118

They both mean that the model fits poorly.  Higher R^2 is better; however, it's higher because we have more variability due to adding another variable.

We need to look at adjusted R^2:
old:  0.040
new:  0.024

When we look at the adjusted R^2, we can see that the new simulated numbers make the fit worse.

e) Generate another new predictor - you can draw another list of random numbers from the same distribution as above, or you can draw from a different distribution. Add this predictor to the model in part d). What happends to the $R^2$? Does this mean that the new predictor is useful for predicting laptop prices?

In [13]:
sim2 = numpy.random.poisson(254, 63)


# X3 = X2
X3['sim2'] = sim2
# print(type(X2))
y3 = laptops['Price ($)']

model3 = sm.OLS(y3, X3)
results3 = model3.fit()
results3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Price ($)   R-squared:                       0.204
Model:                            OLS   Adj. R-squared:                  0.102
Method:                 Least Squares   F-statistic:                     2.011
Date:                Thu, 06 Oct 2016   Prob (F-statistic):             0.0700
Time:                        13:51:52   Log-Likelihood:                -474.74
No. Observations:                  63   AIC:                             965.5
Df Residuals:                      55   BIC:                             982.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [95.0% Conf. Int.]
-------------------------------------------------------------------------------------
const             -2526.3269   2315.142     -1.091      0.280     -7165.976  2113.322
Screen Size (in.)   -59.8996     97.681     -0.613      0.542      -255.657   135.858
Hard drive (GB)       0.6463      0.458      1.411      0.164        -0.272     1.564
RAM Memory (GB)      21.7461     73.337      0.297      0.768      -125.225   168.717
USB Ports          -208.8800    119.474     -1.748      0.086      -448.311    30.551
Weight (oz.)        106.4893    100.426      1.060      0.294       -94.768   307.747
sim                   4.6555      2.085      2.233      0.030         0.477     8.834
sim2                 -2.8440      3.810     -0.747      0.459       -10.479     4.791
==============================================================================
Omnibus:                        6.585   Durbin-Watson:                   1.775
Prob(Omnibus):                  0.037   Jarque-Bera (JB):                6.519
Skew:                           0.788   Prob(JB):                       0.0384
Kurtosis:                       2.947   Cond. No.                     4.36e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.36e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

the simulated values drawn from the poisson distrubition also make for a bad model fit.  

old R^2 was: 0.117
new R^2 is: 0.118
R-squared:	0.131

They both mean that the model fits poorly.  Higher R^2 is better; however, it's higher because we have more variability due to adding another variable.

We need to look at adjusted R^2:
old:  0.040
new:  0.024
R-squared:	0.021

When we look at the adjusted R^2, we can see that the new simulated numbers make the fit worse.

**Question 3**  

Squirt Squad is a cleaning service that sends crews to residential homes on either a once-a-month or twice-a-month schedule, depending on the customer’s preference. The owner would like to predict the amount of time required to clean a house based on the square footage of the house, the total number of rooms in the house, the number of bathrooms it has, the size of the cleaning crew, the frequency of the cleaning schedule, and whether or not the household has children. Data can be found in the tables **`squad`** (containing `squad_id`, `home_id`, `crew` and `freq` (0: once-a-month, 1: twice-a-month); **`squad_homes`** (containing `home_id`, `footage`, `rooms`, `baths` and `children` (Squirt Squad assumes the number of children in a house will never change. BONUS: how would you change the schema to account for the possibility that it will?)); and **`squad_times`** (containing `squad_id` and `dt`, `time` and `crew` (redundant with `squad` but included in case the squad size changes)). You will need to construct a three-way join using `home_id` and `squad_id`.

a) Construct a regression model using all of the independent variables.  

b) Test and interpret the significance of the overall regression model (what is the result of the overall F test)?  

c) Interpret the meaning of the regression coefficient for the Rooms, Crew, Children, and Frequency variables.  

d) Using the p-values, identify which independent variables are significant (needed).  

e) Construct a regression model using only the significant variables found in part d) and predict the average time to clean a house that has 2,250 square feet, 11 total rooms, 3.5 bathrooms, and no children. This house is cleaned once a month with a crew of four employees.  

f) Compare the two models you fitted, which one is a better model? Why?

In [14]:
%pylab inline

import pandas as pd
import statsmodels.api as sm
import yaml

from seaborn import pairplot
from sqlalchemy import create_engine

pg_creds = yaml.load(open('../../pg_creds.yaml'))['student']

engine = create_engine('postgresql://{user}:{password}@{host}:{port}/{dbname}'.format(**pg_creds))

squad = pd.read_sql("SELECT * FROM squad;", engine)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [15]:
squad.head()
squad.describe()

,home_id,crew,freq,squad_id
count,70.000000,70.000000,70.000000,70.000000
mean,34.500000,2.900000,0.485714,35.500000
std,20.351085,0.617381,0.503405,20.351085
min,0.000000,2.000000,0.000000,1.000000
25%,17.250000,3.000000,0.000000,18.250000
50%,34.500000,3.000000,0.000000,35.500000
75%,51.750000,3.000000,1.000000,52.750000
max,69.000000,4.000000,1.000000,70.000000


In [16]:
squad_homes = pd.read_sql("SELECT * FROM squad_homes;", engine)
squad_times = pd.read_sql("SELECT * FROM squad_times;", engine)

In [17]:
squad_homes.head(),squad_homes.describe()


(   home_id  footage  rooms  baths  children
 0        0     1548      8    2.0         0
 1        1     1599      7    1.5         0
 2        2     1630      8    2.0         0
 3        3     1640      7    1.5         0
 4        4     1711      8    2.5         1,
          home_id      footage      rooms      baths   children
 count  70.000000    70.000000  70.000000  70.000000  70.000000
 mean   34.500000  2385.300000  10.871429   3.200000   0.471429
 std    20.351085   422.376175   2.186397   0.873938   0.502787
 min     0.000000  1548.000000   7.000000   1.500000   0.000000
 25%    17.250000  2163.250000   9.000000   2.500000   0.000000
 50%    34.500000  2341.500000  11.000000   3.000000   0.000000
 75%    51.750000  2595.750000  13.000000   4.000000   1.000000
 max    69.000000  3517.000000  15.000000   4.500000   1.000000)

In [18]:
squad_times.head(),squad_times.describe()

(   squad_id         dt  time  crew
 0         1 2016-09-17   132     3
 1         2 2016-09-09   146     2
 2         3 2016-09-12   131     3
 3         4 2016-09-11   141     3
 4         5 2016-09-27   144     3,          squad_id        time        crew
 count  278.000000  278.000000  278.000000
 mean    35.679856  150.017986    2.917266
 std     20.574467   27.716780    0.615964
 min      1.000000   66.000000    2.000000
 25%     18.000000  133.250000    3.000000
 50%     35.500000  150.000000    3.000000
 75%     53.000000  167.000000    3.000000
 max     70.000000  241.000000    4.000000)

In [19]:
squad_times

,squad_id,dt,time,crew
0,1,2016-09-17,132,3
1,2,2016-09-09,146,2
2,3,2016-09-12,131,3
3,4,2016-09-11,141,3
4,5,2016-09-27,144,3
5,6,2016-09-08,162,3
6,7,2016-09-24,140,4
7,8,2016-09-18,162,3
8,9,2016-09-29,138,3
9,10,2016-09-13,165,2


In [20]:
squad_table = pd.read_sql("SELECT distinct squad_id, count(dt) from squad_times group by squad_id;", engine)


In [21]:
squad_data = pd.read_sql('select distinct t.Squad_id, t.dt, t.time, s.home_id, s.crew ,s.freq, h.footage, h.rooms, h.baths, max(h.children) as max_children from squad_times t left join squad s on t.squad_id = s.squad_id left join squad_homes h on s.home_id = h.home_id group by t.Squad_id, t.dt, t.time, t.crew, s.home_id, s.crew ,s.freq, h.footage, h.rooms, h.baths;', engine)

In [22]:
squad_data.head()

,squad_id,dt,time,home_id,crew,freq,footage,rooms,baths,max_children
0,18,2016-07-08,124,17,2,1,2159,9,2.5,0
1,37,2016-08-18,154,36,3,1,2351,13,3.0,0
2,63,2016-08-23,169,62,3,1,3007,13,4.0,0
3,29,2016-09-15,132,28,3,1,2255,10,3.5,0
4,10,2016-08-30,190,9,2,1,2010,7,2.5,1


In [23]:
squad_data.describe()

,squad_id,time,home_id,crew,freq,footage,rooms,baths,max_children
count,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000,278.000000
mean,35.679856,150.017986,34.679856,2.917266,0.611511,2388.823741,10.895683,3.215827,0.428058
std,20.574467,27.716780,20.574467,0.615964,0.488286,434.967464,2.212513,0.880085,0.495690
min,1.000000,66.000000,0.000000,2.000000,0.000000,1548.000000,7.000000,1.500000,0.000000
25%,18.000000,133.250000,17.000000,3.000000,0.000000,2159.000000,9.000000,2.500000,0.000000
50%,35.500000,150.000000,34.500000,3.000000,1.000000,2341.500000,11.000000,3.000000,0.000000
75%,53.000000,167.000000,52.000000,3.000000,1.000000,2597.000000,13.000000,4.000000,1.000000
max,70.000000,241.000000,69.000000,4.000000,1.000000,3517.000000,15.000000,4.500000,1.000000


Regression model:
    
$$\hat{time} = 143.5146  -15.4141 crew  -4.0902 frequency - 0.0048 footage + 6.7580 rooms -5.6075 baths +22.8734 children$$

In [24]:
X5 = squad_data[['crew','freq','footage','rooms','baths','max_children']]
X5 = sm.add_constant(X5)
y5 = squad_data['time']

model5 = sm.OLS(y5,X5)
results5 = model5.fit()
results5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.496
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     44.51
Date:                Thu, 06 Oct 2016   Prob (F-statistic):           1.02e-37
Time:                        13:51:57   Log-Likelihood:                -1222.2
No. Observations:                 278   AIC:                             2458.
Df Residuals:                     271   BIC:                             2484.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
const          143.5146     10.950     13.106      0.000       121.956   165.073
crew           -15.4141      2.026     -7.609      0.000       -19.402   -11.426
freq            -4.0902      2.621     -1.561      0.120        -9.250     1.070
footage         -0.0048      0.006     -0.772      0.441        -0.017     0.007
rooms            6.7580      1.238      5.457      0.000         4.320     9.196
baths           -5.6072      3.247     -1.727      0.085       -12.000     0.786
max_children    22.8734      2.618      8.738      0.000        17.720    28.027
==============================================================================
Omnibus:                        3.347   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.188   Jarque-Bera (JB):                3.313
Skew:                          -0.148   Prob(JB):                        0.191
Kurtosis:                       3.446   Cond. No.                     2.27e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

part b

Ho: All $\beta$ = 0

Ha: At least one $\beta$ not = 0

$\alpha$ = 0.05

Since the p-value of the F-stat (1.02e-37) is less than 0.05, we reject the null hypothesis. At least one of the independent variables is significant to predict time in cleaning a house

c) Interpret the meaning of the regression coefficient for the Rooms, Crew, Children, and Frequency variables.  

    Keeping all other variables constant, increasing the crew size by 1 decreases cleaning time by 15.41 minutes.
    Keeping all other variables constant, increasing the frequency by 1 decreases cleaning time by 4.9 minutes.
    Keeping all other variables constant, increasing the footage size by 1 square foot decreases cleaning time by 0.0048 minutes
    Keeping all other variables constant, increasing the room count by 1 increases cleaning time by 6.758 minutes.
    Keeping all other variables constant, increasing the number of baths by 1 decreases cleaning time by 5.607 minutes.
    Keeping all other variables constant, increasing the number of children by 1 increases cleaning time by 22.8734 minutes.

d) Using the p-values, identify which independent variables are significant (needed).  

Based on the p-values only the coefficients for constant, crew, rooms, children are significant

e) Construct a regression model using only the significant variables found in part d) and predict the average time to clean a house that has 2,250 square feet, 11 total rooms, 3.5 bathrooms, and no children. This house is cleaned once a month with a crew of four employees.  

In [25]:
X6 = squad_data[['crew','rooms','max_children', 'baths']]
X6 = sm.add_constant(X6)
y6 = squad_data['time']

model6 = sm.OLS(y6,X6)
results6 = model6.fit()
results6.summary()

#baths was added after step-wise regression

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   time   R-squared:                       0.491
Model:                            OLS   Adj. R-squared:                  0.483
Method:                 Least Squares   F-statistic:                     65.76
Date:                Thu, 06 Oct 2016   Prob (F-statistic):           6.78e-39
Time:                        13:51:57   Log-Likelihood:                -1223.7
No. Observations:                 278   AIC:                             2457.
Df Residuals:                     273   BIC:                             2476.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
const          137.1231      9.556     14.349      0.000       118.310   155.936
crew           -15.3603      2.008     -7.650      0.000       -19.313   -11.408
rooms            6.4869      1.137      5.703      0.000         4.248     8.726
max_children    24.0843      2.465      9.771      0.000        19.232    28.937
baths           -7.2402      2.836     -2.553      0.011       -12.823    -1.657
==============================================================================
Omnibus:                        4.085   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.130   Jarque-Bera (JB):                4.642
Skew:                          -0.118   Prob(JB):                       0.0982
Kurtosis:                       3.587   Cond. No.                         97.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
print("time it takes to clean house, on average, based on parameters given:",results6.predict([1,4,11,0,3.5]) )

time it takes to clean house, on average, based on parameters given: [ 121.69663826]


f) Compare the two models you fitted, which one is a better model? Why?

The second model is better, because it includes only the variables significant in predicting cleaning time